# BentoML Example: Convolutional Autoencoder with PyTorch



[BentoML](http://bentoml.ai) is an open source framework for building, shipping and running machine learning services. It provides high-level APIs for defining an ML service and packaging its artifacts, source code, dependencies, and configurations into a production-system-friendly format that is ready for deployment.

This notebook demonstrates how to use BentoML to turn a PyTorch model into a docker image containing a REST API server serving this model, how to use your ML service built with BentoML as a CLI tool, and how to distribute it a pypi package.

This example was built based on [this notebook](https://github.com/baldassarreFe/zalando-pytorch).

![Impression](https://www.google-analytics.com/collect?v=1&tid=UA-112879361-3&cid=555&t=event&ec=nb&ea=open&el=official-example&dt=pytorch-fashion-mnist)

In [ ]:
!pip install -I bentoml
!pip install torch torchvision sklearn pillow pandas numpy

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torchvision import transforms
from torch.autograd import Variable

from sklearn.manifold import TSNE
from sklearn.metrics import accuracy_score

import bentoml

## Prepare Dataset


In [ ]:
from torchvision.datasets import FashionMNIST

In [ ]:
import os
import sys
sys.path.append(os.path.join(os.getcwd(), os.pardir, 'src'))

Load train and test set in batches of 1000.

The `28x28` images are scaled up to `29x29` so that combining convolutions and transposed convolutions would not chop off pixels from the reconstructed images.

In [ ]:
batch_size = 1000

transform = transforms.Compose([transforms.CenterCrop((29, 29)), transforms.ToTensor()])
train_dataset = FashionMNIST(
    './data', train=True, download=True, 
    transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size)

test_dataset = FashionMNIST(
    './data', train=False, download=True, 
    transform=transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size)

## Unsupervised reconstruction


In [ ]:
class Encoder(nn.Module):
    def __init__(self, embedding_size):
        super(Encoder, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5, stride=2)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5, stride=2)
        self.conv3 = nn.Conv2d(20, 40, kernel_size=5, stride=2)
        self.fully = nn.Linear(40, embedding_size)

    def forward(self, x):
        # 1x29x29
        x = F.relu(self.conv1(x))
        # 10x13x13
        x = F.relu(self.conv2(x))
        # 20x5x5
        x = F.relu(self.conv3(x))
        # 40x1x1
        x = x.view(x.data.shape[0], 40)
        # 40
        x = self.fully(x)
        # output_size
        return x

class Decoder(nn.Module):
    def __init__(self, input_size):
        super(Decoder, self).__init__()
        self.fully = nn.Linear(input_size, 40)
        self.conv1 = nn.ConvTranspose2d(40, 20, kernel_size=5, stride=2)
        self.conv2 = nn.ConvTranspose2d(20, 10, kernel_size=5, stride=2)
        self.conv3 = nn.ConvTranspose2d(10, 1, kernel_size=5, stride=2)
    
    def forward(self, x):
        x = self.fully(x)
        x = x.view(x.data.shape[0], 40, 1, 1)
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.sigmoid(self.conv3(x))
        return x

In [ ]:
embedding_size = 20
encoder = Encoder(embedding_size)
decoder = Decoder(embedding_size)

autoencoder = nn.Sequential(encoder, decoder)

## Supervised classification

In [ ]:
for param in encoder.parameters():
    param.requires_grad = False

classifier = nn.Sequential(
    encoder, 
    nn.Linear(embedding_size, 15),
    nn.ReLU(),
    nn.Linear(15, len(FashionMNIST.classes)),
    nn.LogSoftmax()
)

## Model Training

In [ ]:
classifier.train()

loss_fn = nn.NLLLoss()
optimizer = optim.Adam([p for p in classifier.parameters() if p.requires_grad])
epoch_loss = []

for epoch in range(2):
    batch_loss = []
    for batch_num, (data, targets) in enumerate(train_loader):
        data, targets = Variable(data), Variable(targets)
        optimizer.zero_grad()
        output = classifier(data)
        loss = loss_fn(output, targets)
        loss.backward()
        optimizer.step()
        batch_loss.append(loss.data.item() )
    epoch_loss.append(sum(batch_loss) / len(batch_loss))
    accuracy = accuracy_score(targets.data.numpy(), output.data.numpy().argmax(axis=1))
    print('Epoch {}:\tloss {:.4f}\taccuracy {:.2%}'.format(epoch, epoch_loss[-1], accuracy))

## Evaluation

In [ ]:
classifier.eval()
data, targets = next(test_loader.__iter__())

outputs = classifier(Variable(data))
log_probs, output_classes = outputs.max(dim=1)

accuracy = accuracy_score(targets.numpy(), output_classes.data.numpy())
print('Accuracy: {:.2%}'.format(accuracy))

In [ ]:
fig, axex = plt.subplots(8, 8, figsize=(16, 16))

zip_these = axex.ravel(), log_probs.data.exp(), output_classes.data, targets, data.numpy().squeeze()
for ax, prob, output_class, target, img in zip(*zip_these):
    ax.imshow(img, cmap='gray' if output_class == target else 'autumn')
    ax.axis('off')
    ax.set_title('{} {:.1%}'.format(FashionMNIST.classes[output_class], prob))

# Define ML service with BentoML

In [ ]:
%%writefile pytorch_fashion_mnist.py

import torch
import bentoml
from PIL import Image
from torchvision import transforms

from bentoml.artifact import PytorchModelArtifact
from bentoml.handlers import DataframeHandler
from torch.autograd import Variable
from torchvision.datasets import FashionMNIST

@bentoml.env(conda_dependencies=['torch', 'numpy', 'torchvision', 'scikit-learn'])
@bentoml.artifacts([PytorchModelArtifact('classifier')])
class FashionMNISTModel(bentoml.BentoService):
    """
    documentation strip
    """

    @bentoml.api(DataframeHandler)
    def predict(self, df):
        tensor = torch.tensor(df.values, dtype=torch.uint8)
        img = Image.fromarray(tensor.numpy(), mode='L')
        transform = transforms.Compose([transforms.CenterCrop((29, 29)), transforms.ToTensor()])
        img = transform(img)

        outputs = self.artifacts.classifier(Variable(torch.tensor([img.numpy()])))
        _, output_classes = outputs.max(dim=1)


        return FashionMNIST.classes[output_classes]

# Save BentoML service archive

In [ ]:
from pytorch_fashion_mnist import FashionMNISTModel
new_model = FashionMNISTModel.pack(classifier=classifier)


saved_path = new_model.save('/tmp/bento')
print(saved_path)

# Load from BentoML service archive

### Run REST API server

In [ ]:
!bentoml serve {saved_path}

### Call REST API from a client

Run the following command on a command line terminal:

curl -X POST 'http://127.0.0.1:5000/predict' -d '[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 13, 67, 0, 0, 0, 0, 50, 38, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 8, 120, 209, 226, 247, 237, 255, 255, 255, 247, 238, 235, 172, 72, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 137, 239, 252, 243, 234, 229, 238, 244, 246, 240, 230, 232, 239, 248, 251, 194, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 102, 255, 231, 228, 227, 228, 233, 230, 230, 229, 228, 232, 232, 231, 227, 224, 252, 179, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 233, 241, 229, 231, 255, 255, 238, 231, 227, 238, 246, 228, 230, 227, 234, 235, 229, 241, 20, 0, 0, 0, 0], [0, 0, 0, 0, 0, 248, 241, 231, 255, 149, 47, 252, 228, 255, 242, 216, 238, 232, 255, 228, 220, 234, 250, 54, 0, 0, 0, 0], [0, 0, 0, 0, 0, 255, 240, 232, 255, 15, 0, 255, 237, 191, 0, 0, 214, 255, 13, 123, 255, 234, 252, 114, 0, 0, 0, 0], [0, 0, 0, 0, 6, 255, 238, 239, 255, 177, 0, 255, 255, 0, 130, 116, 47, 65, 43, 37, 255, 236, 249, 162, 0, 0, 0, 0], [0, 0, 0, 0, 32, 255, 236, 245, 255, 204, 0, 255, 84, 0, 37, 28, 31, 0, 25, 13, 255, 236, 249, 199, 0, 0, 0, 0], [0, 0, 0, 0, 53, 255, 236, 250, 250, 231, 2, 255, 21, 0, 221, 255, 236, 54, 245, 198, 243, 238, 245, 223, 0, 0, 0, 0], [0, 0, 0, 0, 80, 255, 237, 250, 240, 255, 0, 0, 39, 157, 0, 0, 215, 94, 20, 126, 255, 237, 239, 250, 0, 0, 0, 0], [0, 0, 0, 0, 101, 255, 235, 253, 244, 243, 133, 138, 208, 255, 201, 214, 255, 230, 7, 174, 255, 240, 238, 255, 0, 0, 0, 0], [0, 0, 0, 0, 126, 255, 233, 255, 248, 233, 255, 255, 240, 232, 243, 243, 231, 251, 255, 255, 254, 243, 238, 255, 3, 0, 0, 0], [0, 0, 0, 0, 147, 255, 233, 249, 181, 243, 227, 224, 230, 234, 230, 230, 235, 228, 235, 222, 207, 255, 236, 255, 35, 0, 0, 0], [0, 0, 0, 0, 163, 255, 245, 221, 86, 255, 233, 233, 235, 236, 234, 234, 234, 232, 242, 231, 125, 255, 236, 255, 55, 0, 0, 0], [0, 0, 0, 0, 181, 254, 255, 200, 69, 255, 228, 232, 234, 235, 234, 234, 233, 235, 241, 237, 70, 255, 235, 246, 57, 0, 0, 0], [0, 0, 0, 0, 197, 247, 255, 188, 110, 255, 224, 233, 234, 234, 234, 234, 234, 234, 240, 253, 69, 255, 236, 248, 77, 0, 0, 0], [0, 0, 0, 0, 200, 246, 255, 149, 145, 255, 223, 235, 234, 235, 235, 235, 234, 237, 233, 255, 47, 255, 239, 249, 98, 0, 0, 0], [0, 0, 0, 0, 204, 243, 255, 111, 173, 255, 227, 235, 235, 236, 235, 235, 235, 239, 229, 255, 19, 227, 246, 249, 110, 0, 0, 0], [0, 0, 0, 0, 196, 240, 255, 109, 213, 250, 229, 235, 235, 236, 235, 237, 236, 237, 226, 255, 55, 203, 251, 245, 120, 0, 0, 0], [0, 0, 0, 0, 192, 243, 255, 114, 232, 240, 232, 235, 235, 236, 234, 237, 236, 235, 229, 255, 134, 171, 252, 244, 137, 0, 0, 0], [0, 0, 0, 0, 189, 251, 255, 154, 238, 233, 236, 234, 235, 236, 235, 238, 236, 235, 232, 255, 166, 125, 255, 243, 142, 0, 0, 0], [0, 0, 0, 0, 183, 252, 255, 171, 247, 232, 234, 234, 233, 233, 232, 234, 233, 234, 233, 240, 223, 128, 255, 242, 151, 0, 0, 0], [0, 0, 0, 0, 178, 243, 255, 57, 238, 241, 238, 238, 238, 237, 236, 237, 237, 240, 237, 254, 176, 52, 255, 239, 157, 0, 0, 0], [0, 0, 0, 0, 188, 240, 250, 62, 218, 255, 220, 222, 222, 222, 223, 223, 224, 222, 218, 255, 154, 32, 255, 236, 188, 0, 0, 0], [0, 0, 0, 0, 130, 245, 242, 24, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 34, 242, 244, 135, 0, 0, 0], [0, 0, 0, 0, 76, 255, 249, 22, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 39, 249, 255, 123, 0, 0, 0], [0, 0, 0, 0, 49, 205, 197, 11, 0, 4, 1, 2, 2, 2, 2, 2, 2, 2, 2, 3, 0, 3, 174, 189, 67, 0, 0, 0]]' -H 'Content-Type: application/json'

You will see the predication result of your model.

# Run REST API server with Docker

** _Note: `docker` is not available when running in Google Colaboratory_

In [ ]:
!cd {saved_path} && docker build . -t pytorch-fashion-mnist

In [ ]:
!docker run -p 5000:5000 pytorch-fashion-mnist